<a href="https://colab.research.google.com/github/chathasphere/chathasphere.github.io/blob/main/teaching/306_materials/003_lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 8: Regular Expressions and Strings
## March 22nd, 2022

In [1]:
require(tidyverse)
require(stringr)

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# 1. Regular Expressions

Regular expressions (regex) are a way of describing **patterns** in text. In practice, they are used to search for (and/or replace) substrings. They can be trickyL

`Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems.` - Jamie Zawinski

As a concrete example, suppose that we want to find and extract all the email addresses in a document. Do we manually search for all email addresses? Or do email addresses follow an abstract pattern that we can capture automatically?

## 1.1 Special characters

Regex takes advantage of several reserved characters that are used for special functions. 

`. \ | ( ) [ ] ^ $ { } * + ?`

We'll go through a bunch of definitions. These will make more sense with some concrete examples later on. For more practice, you can check out https://regexone.com/.

### Character classes

* `.` matches anything (wildcard)
* `[aeiou]` matches a single character in the set provided
* `[^aeiou]` matches a single character NOT in the set
* `[a-e]` matches a single character in a range, equivalent to `[abcde]`

#### Shorthand

* `\w` matches a "word" character, equivalent to `[a-zA-Z0-9]`
* `\s` matches any whitespace, including tabs and newlines
* `\d` matches digits, equivalent to `[0-9]`
* `\W`, `\S`, and `\D` match the opposite of the lower-case versions

#### Special characters

* Note that `\t` and `\n` match the tab and newline characters. 
* If you want the "literal" versions of any of the reserved characters, (e.g., if we want to match the period ".") you will need to escape them with a backslash `\`, e.g. `[\.\\\|]`


### Grouping

* `()` are used to group patterns together. This can be used with any of the below operators. This can also be used to extract portions of a regex out individually, which we will later learn.
* `\1`, `\2`, etc. refers to the first, second, etc. group in the match.

### Operators

* `|` is the OR operator and allows matches of either side
* `{}` describes how many times the preceeding character of group must occur:
  * `{m}` must occur exactly `m` times
  * `{m,n}` must occur between `m` and `n` times, inclusive
  * `{m,}` Must occur at least `m` times
* `*` means the preceeding character can appear zero or more times, equivalent to `{0,}`
* `+` means the preceeding character must appear one or more times, equivalent to `{1,}`
* `?` means the preceeding character can appear zero or one time, equivalent to `{0,1}`

### Anchors

* `^` matches the start of a string (or line)
* `$` matches the end of a string (or line)
* `\b` matches a word "boundary"
* `\B` matches not word boundary

# 2. Handling Strings in R

You're already familiar with the basics of strings. Note that there are some special characters. The most commonly used ones are `\n` and `\t` for newlines and tabs, respectively.

Also note that there are some reserved characters do special things in strings. If you want to include them, you must escape them with a backslash `\`.

In [4]:
double_quote = "hi\"bye"
backslash_ex = "a\\tb"
backslash_ex2 = "a\tb"

Running `print(double_quote)` shows the unformatted string--we'll want to use the `cat` function instead. 'cat' means "concatenate and print."

In [ ]:
cat(double_quote)

hi"bye

In [ ]:
cat(backslash_ex)

a\tb

In [ ]:
cat(backslash_ex2)

a	b

You’ll also sometimes see strings like `"\u00b5"`($\mu$), this is called Unicode-escaping, and is a way of writing non-ASCII characters that works on all platforms.

In [ ]:
cat("\u00b5")

µ

In [ ]:
cat("\u00e7 (c-cedilla) is a Latin script letter, used in the Albanian, Azerbaijani, Manx, Tatar, Turkish, Turkmen, Kurdish, Zazaki, and Romance alphabets." )


ç (c-cedilla) is a Latin script letter, used in the Albanian, Azerbaijani, Manx, Tatar, Turkish, Turkmen, Kurdish, Zazaki, and Romance alphabets.

In [ ]:
cat("You can even use emojis like: \U0001f637")

You can even use emojis like: 😷

### String Functions

In [10]:
ne_states <- c("Connecticut", "Maine", "Massachusetts", "Vermont", "New Hampshire", "Rhode Island")

In [17]:
# measure the lengths of strings within a chr vector
str_length(ne_states)

[1] 11  5 13  7 13 12

In [15]:
# string analog of `c`
str_c("lions", "tigers", "bears", "oh my!")

[1] "lionstigersbearsoh my!"

In [18]:
cat(str_c('Istanbul', 'Turkey\n', sep=', '))
cat(str_c('Ann Arbor', 'MI', "USA", sep=', '))

Istanbul, Turkey
Ann Arbor, MI, USA

In [21]:
# what happens when we combine str_c with c?
vec <- c("a", "b", "c")
str_c("d", vec)

[1] "da" "db" "dc"

In [22]:
x = c('abc', '123', NA)
str_c('|-', x, '-|')

[1] "|-abc-|" "|-123-|" NA

In [24]:
str_c('|-', str_replace_na(x, "UNK"), '-|') # finds NA and replaces with 'UNK'

[1] "|-abc-|" "|-123-|" "|-UNK-|"

To collapse a vector of strings, use the `collapse` argument to `str_c`:

In [25]:
str_c(ne_states, collapse=", ")

[1] "Connecticut, Maine, Massachusetts, Vermont, New Hampshire, Rhode Island"

### Subsetting Strings

In [ ]:
ne_states = c("Connecticut", "Maine", "Massachusetts", "Vermont", "New Hampshire", "Rhode Island")
ne_states

[1] "Connecticut"   "Maine"         "Massachusetts" "Vermont"      
[5] "New Hampshire" "Rhode Island"

In [29]:
# selects first 3 characters in each
str_sub(ne_states, 1, 3)

[1] "Con" "Mai" "Mas" "Ver" "New" "Rho"

In [ ]:
# selects last 3 characters
str_sub(ne_states, -3, -1)

[1] "cut" "ine" "tts" "ont" "ire" "and"

In [30]:
str_sub(ne_states, 1, 7)  # Maine is 5 letters, but this worked still

[1] "Connect" "Maine"   "Massach" "Vermont" "New Ham" "Rhode I"

In [31]:
# select and mutate substrings
str_sub(ne_states, 1, 1) <- str_to_lower(str_sub(ne_states, 1, 1))
# this alters the original string!
ne_states

[1] "connecticut"   "maine"         "massachusetts" "vermont"      
[5] "new Hampshire" "rhode Island"

In [ ]:
str_sub(ne_states, -3, -1) <- str_to_upper(str_sub(ne_states, -3, -1))
ne_states

[1] "connectiCUT"   "maINE"         "massachuseTTS" "vermONT"      
[5] "new HampshIRE" "rhode IslAND"

### String Replacement
We have seen this in previous labs, but a quick review...

In [34]:
str_replace("dragonfly", "fly", "")

[1] "dragon"

In [35]:
str_replace_all("banana", "a", "o")

[1] "bonono"

# 3. RegEx in R

In `R`, we will use  `str_detect` and `str_extract` (or `str_extract_all`) to play wtih regular expressions.

In [38]:
x = c("apple", "banana", "pear", "orange")

In [39]:
str_detect(x, "an")

[1] FALSE  TRUE FALSE  TRUE

In [40]:
str_extract(x, "an")

[1] NA   "an" NA   "an"

In [37]:
baseball = "According to Baseball Reference’s wins above average, The Red Sox had the best 
outfield in baseball— one-tenth of a win ahead of the Milwaukee Brewers, 11.5 to 11.4. And 
that’s despite, I’d argue, the two best position players in the NL this year (Christian 
Yelich and Lorenzo Cain) being Brewers outfielders. More importantly, the distance from 
Boston and Milwaukee to the third-place Yankees is about five wins. Two-thirds of the Los 
Angeles Angels’ outfield is Mike Trout (the best player in baseball) and Justin Upton (a 
four-time All-Star who hit 30 home runs and posted a 122 OPS+ and .348 wOba this year), 
and in order to get to 11.5 WAA, the Angels’ outfield would have had to replace right 
fielder Kole Calhoun with one of the three best outfielders in baseball this year by WAA."

#### 1 Write a regex that captures all capitalized words.

In [43]:
str_extract_all(baseball, "\\b[A-Z][a-z]+") #1 Write a regex that captures all capitalized words.
# think for a second: why do we need two backslashes to begin with?

[[1]]
 [1] "According" "Baseball"  "Reference" "The"       "Red"       "Sox"      
 [7] "Milwaukee" "Brewers"   "And"       "Christian" "Yelich"    "Lorenzo"  
[13] "Cain"      "Brewers"   "More"      "Boston"    "Milwaukee" "Yankees"  
[19] "Two"       "Los"       "Angeles"   "Angels"    "Mike"      "Trout"    
[25] "Justin"    "Upton"     "All"       "Star"      "Angels"    "Kole"     
[31] "Calhoun"

Breaking down the above Regex:
- `\b` looks for a word boundary (not just the beginning of the text snippet!)
- `[A-Z]` matches a single capitalized letter
- `[a-z]` matches a single lowercase letter
- `+` means we match arbitrarily many lowercase letters

#### 2 Write a regex that captures all the numbers

In [45]:
str_extract_all(baseball, "\\.?\\d+\\.?\\d*") 
# exercise: break down what the component parts of this Regex are doing

[[1]]
[1] "11.5" "11.4" "30"   "122"  ".348" "11.5"

#### 3 Write a regex that captures all hyphenated words

In [46]:
str_extract_all(baseball, "\\w+-\\w+") # \w stands for an arbitrary letter

[[1]]
[1] "one-tenth"   "third-place" "Two-thirds"  "four-time"   "All-Star"

#### 4 Write a regex that captures all words with two consecutive wovels

In [ ]:
str_extract_all(baseball, "\\w*[aeiou]{2}\\w*")

[[1]]
 [1] "outfield"    "ahead"       "Milwaukee"   "argue"       "position"   
 [6] "year"        "Christian"   "Cain"        "being"       "outfielders"
[11] "Milwaukee"   "Yankees"     "about"       "outfield"    "Trout"      
[16] "four"        "year"        "outfield"    "would"       "fielder"    
[21] "Calhoun"     "three"       "outfielders" "year"

#### 5 Write a regex that captures all words with a repeated letter

In [56]:
str_extract_all(baseball, "\\w*([a-zA-Z])\\1\\w*")
# equivalently: str_extract_all(baseball, "\\w*([\\w])\\1\\w*")
# the \1 is a backreference that matches the (first and only) () group

[[1]]
 [1] "According" "Baseball"  "baseball"  "Milwaukee" "Milwaukee" "Yankees"  
 [7] "baseball"  "All"       "WAA"       "three"     "baseball"  "WAA"

#### 6 Write a regex that matches "this" and "the" but not "third"

In [ ]:
str_extract_all(baseball, "th(e|is)")
str_extract_all(baseball, "(t|T)h(e|is)") # including capitalized T

[[1]]
 [1] "the"  "the"  "the"  "the"  "this" "the"  "the"  "the"  "the"  "this"
[11] "the"  "the"  "this"

[[1]]
 [1] "The"  "the"  "the"  "the"  "the"  "this" "the"  "the"  "the"  "the" 
[11] "this" "the"  "the"  "this"

Note that any time you want to use a backslash `\` in a regex pattern in `R`, you'll need to use a double backslash `\\` instead. This is because `R` has its own layer of string processing that also uses backslashes to escape reserved characters. So you need to tell `R` to use a literal backslash so that it passes a backslash to the regex function.

In [ ]:
naive = "a.c"
dot = "a\\.c"

cat(naive)
str_detect(c("abc", "a.c", "bef"), naive) # matches anything a-blank-c because . is a wildcard

cat(dot)
str_detect(c("abc", "a.c", "bef"), dot)

a.c

[1]  TRUE  TRUE FALSE

a\.c

[1] FALSE  TRUE FALSE

Question: How many backslashes do you need to create a regex pattern that matches a literal backslash when using `R`?

In [57]:
x = "a\\b"
cat(x)

a\b

In [68]:
str_extract(x, "\\\\")
# remember, the parser interprets each "\\" as '\'.
# hence the raw string "\\\\" becomes the regex "\\" which matches "\" twice.

[1] "\\"

## Exercises

Use `stringr::words` to do the exercises

In [ ]:
words

[1] "a"           "able"        "about"       "absolute"    "accept"     
  [6] "account"     "achieve"     "across"      "act"         "active"     
 [11] "actual"      "add"         "address"     "admit"       "advertise"  
 [16] "affect"      "afford"      "after"       "afternoon"   "again"      
 [21] "against"     "age"         "agent"       "ago"         "agree"      
 [26] "air"         "all"         "allow"       "almost"      "along"      
 [31] "already"     "alright"     "also"        "although"    "always"     
 [36] "america"     "amount"      "and"         "another"     "answer"     
 [41] "any"         "apart"       "apparent"    "appear"      "apply"      
 [46] "appoint"     "approach"    "appropriate" "area"        "argue"      
 [51] "arm"         "around"      "arrange"     "art"         "as"         
 [56] "ask"         "associate"   "assume"      "at"          "attend"     
 [61] "authority"   "available"   "aware"       "away"        "awful"      
 [66] "baby"        "back"        "bad"         "bag"         "balance"    
 [71] "ball"        "bank"        "bar"         "base"        "basis"      
 [76] "be"          "bear"        "beat"        "beauty"      "because"    
 [81] "become"      "bed"         "before"      "begin"       "behind"     
 [86] "believe"     "benefit"     "best"        "bet"         "between"    
 [91] "big"         "bill"        "birth"       "bit"         "black"      
 [96] "bloke"       "blood"       "blow"        "blue"        "board"      
[101] "boat"        "body"        "book"        "both"        "bother"     
[106] "bottle"      "bottom"      "box"         "boy"         "break"      
[111] "brief"       "brilliant"   "bring"       "britain"     "brother"    
[116] "budget"      "build"       "bus"         "business"    "busy"       
[121] "but"         "buy"         "by"          "cake"        "call"       
[126] "can"         "car"         "card"        "care"        "carry"      
[131] "case"        "cat"         "catch"       "cause"       "cent"       
[136] "centre"      "certain"     "chair"       "chairman"    "chance"     
[141] "change"      "chap"        "character"   "charge"      "cheap"      
[146] "check"       "child"       "choice"      "choose"      "Christ"     
[151] "Christmas"   "church"      "city"        "claim"       "class"      
[156] "clean"       "clear"       "client"      "clock"       "close"      
[161] "closes"      "clothe"      "club"        "coffee"      "cold"       
[166] "colleague"   "collect"     "college"     "colour"      "come"       
[171] "comment"     "commit"      "committee"   "common"      "community"  
[176] "company"     "compare"     "complete"    "compute"     "concern"    
[181] "condition"   "confer"      "consider"    "consult"     "contact"    
[186] "continue"    "contract"    "control"     "converse"    "cook"       
[191] "copy"        "corner"      "correct"     "cost"        "could"      
[196] "council"     "count"       "country"     "county"      "couple"     
[201] "course"      "court"       "cover"       "create"      "cross"      
[206] "cup"         "current"     "cut"         "dad"         "danger"     
[211] "date"        "day"         "dead"        "deal"        "dear"       
[216] "debate"      "decide"      "decision"    "deep"        "definite"   
[221] "degree"      "department"  "depend"      "describe"    "design"     
[226] "detail"      "develop"     "die"         "difference"  "difficult"  
[231] "dinner"      "direct"      "discuss"     "district"    "divide"     
[236] "do"          "doctor"      "document"    "dog"         "door"       
[241] "double"      "doubt"       "down"        "draw"        "dress"      
[246] "drink"       "drive"       "drop"        "dry"         "due"        
[251] "during"      "each"        "early"       "east"        "easy"       
[256] "eat"         "economy"     "educate"     "effect"      "egg"        
[261] "eight"       "either"      "elect"       "e

### 1. Which words start with `y`? (Freebie)

In [ ]:
str_extract(words, "^y\\w*")

[1] NA          NA          NA          NA          NA          NA         
  [7] NA          NA          NA          NA          NA          NA         
 [13] NA          NA          NA          NA          NA          NA         
 [19] NA          NA          NA          NA          NA          NA         
 [25] NA          NA          NA          NA          NA          NA         
 [31] NA          NA          NA          NA          NA          NA         
 [37] NA          NA          NA          NA          NA          NA         
 [43] NA          NA          NA          NA          NA          NA         
 [49] NA          NA          NA          NA          NA          NA         
 [55] NA          NA          NA          NA          NA          NA         
 [61] NA          NA          NA          NA          NA          NA         
 [67] NA          NA          NA          NA          NA          NA         
 [73] NA          NA          NA          NA          NA          NA         
 [79] NA          NA          NA          NA          NA          NA         
 [85] NA          NA          NA          NA          NA          NA         
 [91] NA          NA          NA          NA          NA          NA         
 [97] NA          NA          NA          NA          NA          NA         
[103] NA          NA          NA          NA          NA          NA         
[109] NA          NA          NA          NA          NA          NA         
[115] NA          NA          NA          NA          NA          NA         
[121] NA          NA          NA          NA          NA          NA         
[127] NA          NA          NA          NA          NA          NA         
[133] NA          NA          NA          NA          NA          NA         
[139] NA          NA          NA          NA          NA          NA         
[145] NA          NA          NA          NA          NA          NA         
[151] NA          NA          NA          NA          NA          NA         
[157] NA          NA          NA          NA          NA          NA         
[163] NA          NA          NA          NA          NA          NA         
[169] NA          NA          NA          NA          NA          NA         
[175] NA          NA          NA          NA          NA          NA         
[181] NA          NA          NA          NA          NA          NA         
[187] NA          NA          NA          NA          NA          NA         
[193] NA          NA          NA          NA          NA          NA         
[199] NA          NA          NA          NA          NA          NA         
[205] NA          NA          NA          NA          NA          NA         
[211] NA          NA          NA          NA          NA          NA         
[217] NA          NA          NA          NA          NA          NA         
[223] NA          NA          NA          NA          NA          NA         
[229] NA          NA          NA          NA          NA          NA         
[235] NA          NA          NA          NA          NA          NA         
[241] NA          NA          NA          NA          NA          NA         
[247] NA          NA          NA          NA          NA          NA         
[253] NA          NA          NA          NA          NA          NA         
[259] NA          NA          NA          NA          NA          NA         
[265] NA          NA          NA          NA          NA          NA         
[271] NA          NA          NA          NA          NA          NA         
[277] NA          NA          NA          NA          NA          NA         
[283] NA          NA          NA          NA          NA          NA         
[289] NA          NA          NA          NA          NA          NA         
[295] NA          NA          NA          NA          NA          NA         
[301] NA          NA          NA          NA          NA          NA         
[307] NA          NA    

In [ ]:
na.omit(str_extract(words, "^y\\w*"))

[1] "year"      "yes"       "yesterday" "yet"       "you"       "young"    
attr(,"na.action")
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
[271] 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
[289] 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
[307] 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
[325] 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
[343] 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360
[361] 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378
[379] 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396
[397] 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414
[415] 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432
[433] 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450
[451] 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468
[469] 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486
[487] 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504
[505] 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522
[523] 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540
[541] 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558
[559] 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576
[577] 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594
[595] 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612
[613] 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630
[631] 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648
[649] 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666
[667] 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684
[685] 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702
[703] 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720
[721] 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738
[739] 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756
[757] 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774
[775] 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792
[793] 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810
[811] 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828
[829] 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846
[847] 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864
[865] 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882
[883] 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900
[901]

### 2. Which words end with `x`?

### 3. ...are exactly two letters long (don’t use `str_length` here)?

### 4. ...have ten or more letters?

### 5. ...end with `ed`, but not with `eed`?

### 6. ...end with `ing` or `ise`?

### 7. ...end with the same two-letter sequence they start with (e.g. `church`)?

### 8 Try to match the valid `dates` below (first row) without matching the invalid dates (the latter six rows).
Hint: Start by writing a pattern that matches all the entries. Then try to refine your pattern to omit the invalid dates.

In [ ]:
dates = c('2012-05-13', '2014-12-31', '1991-06-14', '1991/06/14',
          '200a-05-13',  # invalid year
          '2014-15-20',  # invalid month
          '2014-00-20',  # invalid month
          '2016-04-35',  # invalid day
          '2014-12-00',  # invalid day
          '2013/03-25')  # non-matching separators